Import libary

In [1]:
import pandas as pd
import tensorflow

import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, RepeatVector, TimeDistributed, Input, Activation, Lambda, Flatten
from keras.callbacks import ModelCheckpoint


2025-02-27 13:36:32.124590: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 13:36:32.144661: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 13:36:32.216968: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740638192.347073   20922 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740638192.384134   20922 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 13:36:32.492977: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Convention
Vector: 1D
Matrix: 2D
Tensor: 3D



# Example of dataset read by pandas

Plain Text	Key	Cipher Text	Encrypted Text

taeniform	SECRET	SGSOLBQEKZ	TAENIFORM

fitting	SECRET	BLCZEMUA	FITTING


In [20]:
EPORCHS = 10
LSTM_N_UNITS = 32
SIZE = 10000
TRAIN_SIZE = 10000 * 0.8
TEST_SIZE = 10000 * 0.2


# Maximize number of token in each vector
# => Convention fix in 40 character for input and 8 character for key
MAX_DECRYPT_SEQUENCES_LEN = 100 # Number of word, character to decrypt word
MAX_KEY_SEQUENCES_LEN = 8


In [4]:
def load_data(filename , feature_cols, label_col: str):
    df = pd.read_excel(filename)
    return df[feature_cols], df[label_col]




def tokenize_normalize(tokenizer, maxlen, corpus):
    """
    Perform tokenizer and padding message
    Required tokenizer fit on corpus before proccessing this action
    Example:
        tokenizer = Tokenizer(char_level = True)
        tokenizer.fit_on_texts(features)

    Padding ensure all sequence same length, so maxlen have higher length
    text length in corpus to garuantee not loss of data.

    Return tuple of tokenizered_corpus and padded_corpus
    """
    tokenizered_corpus = tokenizer.texts_to_sequences(corpus)
    padded_corpus = pad_sequences(tokenizered_corpus, maxlen= maxlen, padding="post", truncating="post")
    return tokenizered_corpus, padded_corpus
def tensor_post_proccess(tensor):
    """
    Assume prediction value will have shape (None, 8)
    Normalize to (None, 8, 1), round floating point convert to int
    Prediction will not follow original style, we need to add more demension to decode
    """
    # seq = np.expand_dims(seq, axis = 2)
    normal_tensor = np.argmax(tensor, axis=-1)
    # matrix = one_hot_decode(normal_tensor)


    return normal_tensor


def decode_seq(matrix, tokenizer):
    """
    Assume input shape is (8, 1) from one vector
    Recontructure origin text base on input seq vector
    """
    return "".join(tokenizer.sequences_to_texts(matrix)).strip()
def detokenizer(matrix, tokenizer):
    return [word.replace(" ", "") for word in  tokenizer.sequences_to_texts(matrix)]



In [8]:
!wget "https://github.com/trung-kieen/playfair-key-prediction/raw/refs/heads/main/src/PLAYFAIR_CIPHER_DATASET_RANDOM_KEY_10000.xlsx"

--2025-02-27 13:33:11--  https://github.com/trung-kieen/playfair-key-prediction/raw/refs/heads/main/src/PLAYFAIR_CIPHER_DATASET_RANDOM_KEY_10000.xlsx


Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/trung-kieen/playfair-key-prediction/refs/heads/main/src/PLAYFAIR_CIPHER_DATASET_RANDOM_KEY_10000.xlsx [following]
--2025-02-27 13:33:12--  https://raw.githubusercontent.com/trung-kieen/playfair-key-prediction/refs/heads/main/src/PLAYFAIR_CIPHER_DATASET_RANDOM_KEY_10000.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8003::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1190938 (1,1M) [application/octet-stream]
Saving to: ‘PLAYFAIR_CIPHER_DATASET_RANDOM_KEY_10000.xlsx.1’

PLAYFAIR_CIPHER_DAT 100%[===================>]  

In [5]:
filename = "PLAYFAIR_CIPHER_DATASET_RANDOM_KEY_10000.xlsx"
feature_names= "Encrypted Text"
label_name = "Key"
features, labels = load_data(filename, feature_names, label_name)


Encrypted text and key for encrypt those text are used as features and labels for training

In [6]:
pd.DataFrame(zip(features, labels))

,0,1
0,QHUNPTSDREBLAUCFODBEATYLVOCATIPSLZLY,NONSP
1,RAPRYRMZIDALRPSNAIODIDESCILXYLNCYNANTHAEPY,PORRE
2,WOOMFEKEOMGICSATRTHIERCIFORMSOINRNUNSWSEET,ALLOM
3,DBADSMOSINCUFBAPVERCQMPTRHLZLMOXOTQWLORONAIASERA,NONSP
4,MEGAMFPYRWCKOKHNAXAEROSCOPVXENENATIONX,ALLOM
...,...,...
9995,QSOAUAINCBEAMZINIFPAANHEHLOSISBALXHLUNITEX,PORRE
9996,SIVPHPQUEZADSTIWGVLGYEGAUPQNINLSNTIONALX,PHOTO
9997,EASTERNISMACROPTSXLZHYKTROBTUQIXVEPLIABLERBDEW...,NONSP
9998,INSPIRITMENTFROUALESYNTACTVIPRCNORCENLONQTKAEM...,CALEN


Prepare tokenizer for character encoding

In [7]:
    feature_tokenizer = Tokenizer(char_level=True)
    feature_tokenizer.fit_on_texts(features)

    label_tokenizer = Tokenizer(char_level=True)
    label_tokenizer.fit_on_texts(labels)


Tokenizer will defined how word be encode

In [8]:
feature_tokenizer.word_index

{'e': 1,
 'a': 2,
 'o': 3,
 'i': 4,
 'n': 5,
 'r': 6,
 'l': 7,
 's': 8,
 't': 9,
 'p': 10,
 'c': 11,
 'm': 12,
 'u': 13,
 'h': 14,
 'b': 15,
 'x': 16,
 'y': 17,
 'd': 18,
 'g': 19,
 'q': 20,
 'k': 21,
 'v': 22,
 'f': 23,
 'w': 24,
 'z': 25}

Perform tokenizer and padding text. Result  of this process are vectors with same length

In [9]:
features_padded = tokenize_normalize(feature_tokenizer, MAX_DECRYPT_SEQUENCES_LEN, features)[1]
labels_padded = tokenize_normalize(label_tokenizer, MAX_KEY_SEQUENCES_LEN, labels)[1]


In [10]:
features_padded

array([[20, 14, 13, ...,  0,  0,  0],
       [ 6,  2, 10, ...,  0,  0,  0],
       [24,  3,  3, ...,  0,  0,  0],
       ...,
       [ 1,  2,  8, ...,  0,  0,  0],
       [ 4,  5,  8, ...,  0,  0,  0],
       [ 8,  2,  3, ...,  0,  0,  0]], dtype=int32)

In [11]:
encrypt_vocab_size = len(feature_tokenizer.word_index) + 4
key_vocab_size = len(label_tokenizer.word_index) + 4


In [12]:
X_train, X_test, y_train, y_test = train_test_split(features_padded, labels_padded, test_size=0.2, random_state=42)

Now let define Seq2Seq model

In [13]:
def rnn_machine_translate_model(input_vocab, output_vocab, input_timesteps, output_timesteps, n_units, n_features):

    model = Sequential()
    # model.add(LSTM(units=n_units, return_sequences=True, input_shape=(input_timesteps, 1)))
    model = Sequential()
    model.add(Embedding(input_vocab, n_units, input_length=input_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(output_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(Dense(output_vocab, activation='softmax'))
    model.compile(optimizer="adam",
                  loss='sparse_categorical_crossentropy',
    metrics=[
        tensorflow.keras.metrics.MeanSquaredError(name="mse"),
        tensorflow.keras.metrics.MeanAbsoluteError(name="msa"),
        tensorflow.keras.metrics.BinaryAccuracy(name="accuracy"),
    ]
    )

    return model


In [14]:
model = rnn_machine_translate_model( input_vocab = encrypt_vocab_size, output_vocab = key_vocab_size, input_timesteps= MAX_DECRYPT_SEQUENCES_LEN, output_timesteps= MAX_KEY_SEQUENCES_LEN  ,n_units = LSTM_N_UNITS, n_features = 1)
save_model_file = "translate.keras"
checkpoint = ModelCheckpoint(save_model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


/home/trungkieen/project/playfair-key-prediction/src/env36/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-02-27 13:38:58.651097: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
history = model.fit(X_train, y_train.reshape(y_train.shape[0], y_train.shape[1], 1),
      epochs=EPORCHS, batch_size=1,
      validation_split = 0.2,
      callbacks=[checkpoint], verbose=1)


Epoch 1/10
 566/6400 ━━━━━━━━━━━━━━━━━━━━ 8:25 87ms/step - accuracy: 0.3529 - loss: 1.2089 - msa: 2.9693 - mse: 21.2609

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()


Make prediction

In [ ]:
model = load_model(save_model_file)
preds = model.predict_classes(X_test.reshape((X_test.shape[0],X_test.shape[1])))

In [ ]:
def evaluate(model, X_test, y_test, tokenizer):
    predictions = model.predict(X_test.reshape((X_test.shape[0], X_test.shape[1])), batch_size=64, verbose=0)
    predictions = tensor_post_proccess(predictions)
    # targets = tensor_post_proccess(y_test)

    targets = y_test

    # Reverser tokenizer and remove whitespace
    predictions = detokenizer(predictions, tokenizer)
    targets= detokenizer(y_test ,tokenizer )
    data = dict()
    data["predict"] = predictions
    data["actual"] = targets

    df = pd.DataFrame(data)
    df.to_csv("result.csv")


In [ ]:
evaluate(model, X_test, y_test, label_tokenizer)
